- 성인 인구조사 소득 예측
age: 나이
workclass: 고용 형태
fnlwgt: 사람의 대표성을 나타내는 가중치(final weight)
education: 교육 수준
education.num: 교육 수준 수치
marital.status: 결혼 상태
occupation: 업종
relationship: 가족 관계
race: 인종
sex: 성별
capital.gain: 양도 소득
capital.loss: 양도 손실
hours.per.week: 주당 근무 시간
native.country: 국적
income: 수익 (예측해야 하는 값)

In [2]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../../Datasat/T2-3/adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 15), (6513, 15), (26048, 2), (6513, 2))

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


2. EDA

In [19]:
print(X_train.head())


          id  age  workclass  fnlwgt     education  education.num  \
21851  21851   36    Private  241998     Bachelors             13   
7632    7632   53    Private  103950       Masters             14   
27878  27878   19    Private  203061  Some-college             10   
14121  14121   20    Private  102607       HS-grad              9   
32345  32345   54  State-gov  138852       HS-grad              9   

           marital.status         occupation   relationship   race     sex  \
21851  Married-civ-spouse       Craft-repair        Husband  White    Male   
7632             Divorced     Prof-specialty  Not-in-family  White  Female   
27878       Never-married       Tech-support  Not-in-family  White  Female   
14121       Never-married  Handlers-cleaners      Own-child  White    Male   
32345  Married-civ-spouse     Prof-specialty        Husband  White    Male   

       capital.gain  capital.loss  hours.per.week native.country  
21851             0             0              50

In [20]:
print(y_train.head())

          id income
21851  21851   >50K
7632    7632  <=50K
27878  27878  <=50K
14121  14121  <=50K
32345  32345  <=50K


In [23]:
#타켓수 확인
y_train['income'].value_counts()

income
<=50K    19756
>50K      6292
Name: count, dtype: int64

In [7]:
print(X_train.info())
print(X_test.info())
print(y_train.info())


<class 'pandas.core.frame.DataFrame'>
Index: 26048 entries, 21851 to 25716
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26048 non-null  int64 
 1   age             26048 non-null  int64 
 2   workclass       24592 non-null  object
 3   fnlwgt          26048 non-null  int64 
 4   education       26048 non-null  object
 5   education.num   26048 non-null  int64 
 6   marital.status  26048 non-null  object
 7   occupation      24585 non-null  object
 8   relationship    26048 non-null  object
 9   race            26048 non-null  object
 10  sex             26048 non-null  object
 11  capital.gain    26048 non-null  int64 
 12  capital.loss    26048 non-null  int64 
 13  hours.per.week  26048 non-null  int64 
 14  native.country  25587 non-null  object
dtypes: int64(7), object(8)
memory usage: 3.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 6513 entries, 20901 to 25782
Data columns (t

In [ ]:
# 결측치
# X_train(26048, 15) : workclass(24592, 15), occupation(24585, 15), native.country(25587, 15)

#원핫인코딩(pd.get_dummies) : workclass, education, marital.status, occupation, relationship, race, sex, native.country

In [16]:
print(X_train.describe())


                 id           age        fnlwgt  education.num  capital.gain  \
count  26048.000000  26048.000000  2.604800e+04   26048.000000  26048.000000   
mean   16238.931933     38.610335  1.895741e+05      10.082118   1081.193796   
std     9389.917448     13.628346  1.043848e+05       2.574608   7404.962675   
min        0.000000     17.000000  1.228500e+04       1.000000      0.000000   
25%     8149.500000     28.000000  1.182472e+05       9.000000      0.000000   
50%    16192.500000     37.000000  1.785755e+05      10.000000      0.000000   
75%    24362.250000     48.000000  2.365968e+05      12.000000      0.000000   
max    32560.000000     90.000000  1.484705e+06      16.000000  99999.000000   

       capital.loss  hours.per.week  
count  26048.000000    26048.000000  
mean      88.477695       40.420224  
std      404.689981       12.354707  
min        0.000000        1.000000  
25%        0.000000       40.000000  
50%        0.000000       40.000000  
75%        0.

In [17]:
print(X_test.describe())

                 id          age        fnlwgt  education.num  capital.gain  \
count   6513.000000  6513.000000  6.513000e+03    6513.000000   6513.000000   
mean   16444.247044    38.466912  1.905952e+05      10.074927   1063.471211   
std     9437.636403    13.689111  1.100911e+05       2.565345   7306.639179   
min        5.000000    17.000000  1.930200e+04       1.000000      0.000000   
25%     8116.000000    27.000000  1.166080e+05       9.000000      0.000000   
50%    16602.000000    37.000000  1.775960e+05      10.000000      0.000000   
75%    24663.000000    47.000000  2.400270e+05      12.000000      0.000000   
max    32558.000000    90.000000  1.455435e+06      16.000000  99999.000000   

       capital.loss  hours.per.week  
count   6513.000000     6513.000000  
mean      82.609090       40.506372  
std      395.962575       12.318984  
min        0.000000        1.000000  
25%        0.000000       40.000000  
50%        0.000000       40.000000  
75%        0.000000   

In [25]:
X_train.isnull().sum()


id                   0
age                  0
workclass         1456
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1463
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     461
dtype: int64

In [26]:
X_test.isnull().sum()

id                  0
age                 0
workclass         380
fnlwgt              0
education           0
education.num       0
marital.status      0
occupation        380
relationship        0
race                0
sex                 0
capital.gain        0
capital.loss        0
hours.per.week      0
native.country    122
dtype: int64

In [28]:
X_train['workclass'].value_counts()

workclass
Private             18160
Self-emp-not-inc     2049
Local-gov            1648
State-gov            1037
Self-emp-inc          909
Federal-gov           770
Without-pay            12
Never-worked            7
Name: count, dtype: int64

In [29]:
X_train['occupation'].value_counts()

occupation
Exec-managerial      3323
Prof-specialty       3306
Craft-repair         3296
Adm-clerical         3037
Sales                2898
Other-service        2624
Machine-op-inspct    1584
Transport-moving     1257
Handlers-cleaners    1080
Farming-fishing       786
Tech-support          746
Protective-serv       521
Priv-house-serv       119
Armed-Forces            8
Name: count, dtype: int64

In [30]:
X_train['native.country'].value_counts()

native.country
United-States                 23381
Mexico                          516
Philippines                     158
Germany                         108
Canada                           88
Puerto-Rico                      87
El-Salvador                      76
India                            73
Cuba                             73
England                          69
Italy                            63
South                            62
Jamaica                          59
Vietnam                          57
China                            57
Guatemala                        54
Dominican-Republic               51
Japan                            49
Poland                           47
Columbia                         44
Taiwan                           37
Haiti                            37
Iran                             34
Portugal                         32
Peru                             29
Nicaragua                        27
Ecuador                          25
Greece       

In [31]:
#workclass, native.country는 최빈값, 최빈값이 비슷한 occupation는 null값으로 대체
def data_fillna(df):
  df['workclass'] = df['workclass'].fillna(df['workclass'].mode()[0])
  df['occupation'] = df['occupation'].fillna("null")
  df['native.country'] = df['native.country'].fillna(df['native.country'].mode()[0])
  return df

X_train = data_fillna(df)
X_test = data_fillna(df)

X_train.isnull().sum()
  

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

피처엔지니어링

In [33]:
# 피처 구분
# Numeric features
numeric_features = [
                    'age',
                    'fnlwgt', 
                    'education.num',
                    'capital.gain', 
                    'capital.loss', 
                    'hours.per.week',                     
                  ]

# Categorical features
cat_features = [
                 'workclass',              
                 'education',            
                 'marital.status', 
                 'occupation', 
                 'relationship', 
                 'race', 
                 'sex',
                 'native.country'
                ]

In [34]:
#라벨 인코딩
# from sklearn.preprocessing import LabelEncoder

# all_df = pd.concat([X_train.assign(ind="train"), X_test.assign(ind="test")])
# le = LabelEncoder()
# all_df[cat_features] = all_df[cat_features].apply(le.fit_transform)

# X_train = all_df[all_df['ind'] == 'train']
# X_train = X_train.drop('ind',axis=1)
# X_train

# X_test = all_df[all_df['ind'] == 'test']
# X_test = X_test.drop('ind',axis=1)
# X_test

#원핫 인코딩 (pd.get_dummies)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_train.shape, X_test.shape

((32561, 110), (32561, 110))